## 3. 无重复字符的最长子串

滑窗 + 哈希表

In [37]:
from collections import defaultdict
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        cnt = defaultdict(int)
        left = right = 0
        ans = 0
        while right < len(s):
            # print(cnt)
            cur = s[right]
            cnt[cur] += 1
            while cnt[cur] > 1 and left <= right:
                cnt[s[left]] -= 1
                left += 1
            right += 1
            ans = max(ans, right - left)
        return ans

In [38]:
s = ""

In [39]:
S = Solution()
S.lengthOfLongestSubstring(s)

0

也可以用set代替哈希表的作用

## 206. 反转链表

链表常见双指针

In [40]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [43]:
from typing import Optional
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        pre, cur = None, head
        while cur:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        return pre

### 递归

In [45]:
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        def reverse(pre, cur):
            if not cur: return pre
            nxt = cur.next
            cur.next = pre
            return reverse(cur, nxt)
        return reverse(None, head)

## 146. LRU缓存 (重要)

双向链表 + 哈希表

哈希表中存放双向链表的节点，这样就能以 O(1) 的复杂度进行查询；定位到要操作的节点之后，双链表的增删改都是 O(1) 的复杂度。 同时双向链表中存储键值对，查询映射关系。

In [5]:
# 定义双向链表，并且存储键值对
class DLinkedNode:
    def __init__(self, key = 0, value = 0):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int):
        self.cache = dict()
        self.head = DLinkedNode()
        self.tail = DLinkedNode()  # 伪头部和伪尾节点
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.size = 0
        
    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        # 如果 key 存在，先通过哈希表进行定位，再移动到头部
        node = self.cache[key]
        self.moveToHead(node)
        return node.value
        
    def put(self, key: int, value: int) -> None:
        if key not in self.cache:
            # 如果 key 不存在，创建一个新的节点
            node = DLinkedNode(key, value)
            # 添加进哈希表
            self.cache[key] = node
            # 添加至双向链表的头部
            self.addToHead(node)
            self.size += 1
            if self.size > self.capacity:
                # 如果超出容量，删除双向链表的尾部节点
                removed = self.removeTail()
                # 删除哈希表中对应的项
                self.cache.pop(removed.key)
                self.size -= 1
        else:
            # 如果 key 存在，先通过哈希表定位，再修改 value，并移到头部
            node = self.cache[key]
            node.value = value
            self.moveToHead(node)
    
    def addToHead(self, node):
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node
    
    def removeNode(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev
        
    def moveToHead(self, node):
        self.removeNode(node)
        self.addToHead(node)
    
    def removeTail(self):
        node = self.tail.prev
        self.removeNode(node)
        return node
        
        

In [9]:
class DLinkedNode:
    """双向链表"""
    def __init__(self, key = 0, value = 0):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int):
        self.head = DLinkedNode()
        self.tail = DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.size = 0
        self.cache = dict()
        
    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        node = self.cache[key]
        self.moveToHead(node)
        return node.value
        
    def put(self, key: int, value: int) -> None:
        if key not in self.cache:
            node = DLinkedNode(key, value)
            self.cache[key] = node
            self.addToHead(node)
            self.size += 1
            if self.size > self.capacity:
                removed = self.removeTail()
                self.cache.pop(removed.key)
                self.size -= 1
        else:
            node = self.cache[key]
            node.value = value
            self.moveToHead(node)
    
    def addToHead(self, node):
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node
        
    def removeNode(self, node):
        node.next.prev = node.prev
        node.prev.next = node.next
    
    def moveToHead(self, node):
        self.removeNode(node)
        self.addToHead(node)
        
    def removeTail(self):
        node = self.tail.prev
        self.removeNode(node)
        return node
        
        

## 215. 数组中的第K个最大元素

### 快速选择

In [13]:
import random
from typing import List
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        def partition(nums, l, r):
            cur = random.randint(l, r)
            x, i = nums[cur], l
            nums[cur], nums[r] = nums[r], nums[cur]
            for j in range(l, r):
                if nums[j] <= x:
                    nums[j], nums[i] = nums[i], nums[j]
                    i += 1
            nums[r], nums[i] = nums[i], nums[r]
            return i
        
        def quick_select(nums, l, r, k):
            idx = partition(nums, l, r)
            if idx == k: return nums[idx]
            return quick_select(nums, idx + 1, r, k) if idx < k else\
            quick_select(nums, l, idx - 1, k)
        
        return quick_select(nums, 0, len(nums) - 1, len(nums) - k)

### 最小堆

In [16]:
import heapq
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        heap = []
        for n in nums:
            if len(heap) == k: heapq.heappushpop(heap, n)
            else: heapq.heappush(heap, n)
        return heap[0]